Zadanie polegało na zaimplementowaniu naiwnego klasyfikatora Bayesa dla danych przygotowanych do przetwarzania przez klasyfikator, aby rozpoznawał on czy przychodząca wiadomość jest typu spam lub nie-spam.
Następnie należało dokonać eksperymentów z przygotowanymi zestawami danych: zarówno na pełnych danych, jak dla danych służącymi jako mniejsza część ucząca.
Na koniec należało wyciągnąć wnioski z przeprowadzonych eksperymentów.

Importy:

In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
import warnings

Poniżej klasa implementująca naiwny klasyfikator bayesa.

In [2]:
class NaiveBayes(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.class_labels = None
        self.apriori_labels = None
        self.condition_probs_distributions = None

    def fit(self, x: pd.DataFrame, y: pd.Series):
        self.class_labels = y.unique()

        self.apriori_labels = self.get_apriori_labels(y)
        self.condition_probs_distributions = self.get_condition_probs_distributions(x, y)

    def predict(self, x: pd.DataFrame):
        return self.class_labels[np.argmax(self.predict_proba(x), axis=1)]

    def predict_proba(self, x: pd.DataFrame) -> np.array:
        scores = np.ones((x.shape[0], self.class_labels.size))

        for document_iter, (document_index, document) in enumerate(x.iterrows()):
            for label in self.class_labels:
                for word, count in document.items():
                    if word not in self.condition_probs_distributions.columns:
                        continue

                    value = self.condition_probs_distributions.at[label, word]
                    scores[document_iter, label] += np.log(value) * count

                scores[document_iter, label] += np.log(self.apriori_labels[label])

        return scores

    def get_apriori_labels(self, y: pd.Series) -> pd.Series:
        """
        Calculates how often class occurs in decisions vector (in our default case 1/2 and 1/2).
        """
        apriori_labels = pd.Series([0] * self.class_labels.size)

        for label_iter, label in enumerate(self.class_labels):
            apriori_labels[label_iter] = np.sum(y == label) / y.size

        return apriori_labels

    def get_condition_probs_distributions(self, x: pd.DataFrame, y: pd.Series) -> pd.DataFrame:
        """
        Calculates condition probabilities distributions.
        Frequency calc explanation example:
            * when label == 1 and word == 543 then +1 in [label, word]
        Probs calc explanation example:
            * divide each counts in specific label by sum of words count from all documents
            * before dividing add +1 to counts as laplace fix
        """
        condition_distributions = pd.DataFrame(0, index=self.class_labels, columns=x.columns)

        for document_index, document in x.iterrows():
            label = y[document_index]

            for word, count in document.items():
                condition_distributions.at[label, word] += 1 if count > 0 else 0

        for label in self.class_labels:
            label_count = self.apriori_labels[label] * x.shape[0]
            word_count = x[y == label].applymap(lambda val: 1 if val > 0 else val).sum(axis=0).sum()

            condition_distributions.loc[label] = (condition_distributions.loc[label] + 1) / (word_count + x.shape[1])

        return condition_distributions

Funkcja służąca do wczytania danych oraz przygotowania ich jako macierz, w której wiersze do identyfikatory poszczególnego dokumentu, kolumny to identyfikatory słów występujących w tych dokumentach, a komórka to ilość wystąpień poszczególnego słowa w danym dokumencie.
Funkcja zwraca również klasy decyzyjne dla każdego z dokumentów.

In [3]:
def read_data(size: int = None, data_type: str = 'train'):
    data_size = '' if size is None else f'-{size}'

    data_train = pd.read_csv(f'./DataPrepared/{data_type}-features{data_size}.txt', names=['document', 'word', 'count'], sep=' ')
    data_labels = pd.read_csv(f'./DataPrepared/{data_type}-labels{data_size}.txt', names=['label']).squeeze('columns')
    data_labels.index += 1

    matrix = pd.DataFrame(data=0, index=data_train['document'].unique(), columns=data_train['word'].unique())

    for row_index, row in data_train.iterrows():
        document = row['document']
        word = row['word']

        matrix[word][document] = row['count']

    return matrix, data_labels

Funkcja pomocnicza wykonująca eksperyment z naiwnym klasyfikatorem bayesa dla zadanego rozmiaru danych dla części trenującej. Zwraca ona dokładność otrzymaną przez testowanie zaimplementowanego klasyfikatora.

In [4]:
def experiment_for_size(size: int = None, train_data: tuple = None, test_data: tuple = None):
    matrix_train, data_labels_train = read_data(size) if train_data is None else train_data

    naive_bayes = NaiveBayes()
    naive_bayes.fit(matrix_train, data_labels_train)

    matrix_test, data_labels_test = read_data(data_type='test') if test_data is None else test_data

    return naive_bayes.score(matrix_test, data_labels_test)

Funkcje wykonujące eksperymenty:

In [5]:
def experiment_random():
    matrix_train, data_labels_train = read_data()
    rng = np.random.default_rng()
    random_indexes_to_remove = rng.choice(data_labels_train.size, size=np.random.randint(50), replace=False)

    for random_index in random_indexes_to_remove:
        matrix_train = matrix_train.drop(random_index)
        data_labels_train.drop(random_index)

    score = experiment_for_size(train_data=(matrix_train, data_labels_train))
    print(f'Removed samples (removed documents) from train part:\n{random_indexes_to_remove}')
    print(f'Accuracy for random train size {data_labels_train.size}: {score}')

    
def experiment_with_size():
    sizes = [50, 100, 400, None]
    matrix_test, data_labels_test = read_data(data_type='test')

    for size in sizes:
        score = experiment_for_size(size, test_data=(matrix_test, data_labels_test))
        print(f'Accuracy for train size {size if size is not None else "all"}: {score}')

Wykonanie eksperymentu dla różnych rozmiarów części uczącej:

In [6]:
experiment_with_size()

Accuracy for train size 50: 0.9576923076923077
Accuracy for train size 100: 0.9730769230769231
Accuracy for train size 400: 0.9807692307692307
Accuracy for train size all: 0.9807692307692307


Wnioski do powyższej części:
* Im większa część ucząca, tym lepsza klasyfikacja.
* Nie jest wymagana bardzo duża liczba dokumentów do satysfakcjonującego nauczenia klasyfikatora, aby otrzymać dużą dokładność.
* Przy większej liczbie dokumentów w części uczącej dokładność klasyfikacji jest większa, jednak nie 100%, co oznacza, że wciąż mogą zdarzyć się pojedyncze przypadki, gdzie wiadomość spam zostanie sklasyfikowana jako nie-spam i na odwrót.
* Być może dokładności powinny być większe, jednak może to wynikać z błędów implementacji (prawdopodobnie przy wyliczaniu rozkładu warunkowego wystąpień słów w dokumentach pod warunkiem klasy).

Wykonanie eksperymentu dla losowo usuniętych próbek z części uczącej:

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    experiment_random()

Removed samples (removed documents) from train part:
[612 348 465 585   4 119 402 472 342 174 126 424 504 268 519 354 107 478
 451 622 110 304 220 621 373 483 450 299 186 496 386 554 698  47 297  23
 155  45 356 351 681 103 520 380  56]
Accuracy for random train size 700: 0.9807692307692307


In [12]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    experiment_random()

Removed samples (removed documents) from train part:
[423 264 515 330 169 211 228 411 297  23  80 405 206 385 146 175  18 531
 481 208 681 484 482 430 311 396 101 120 394 347 628 147 651 428]
Accuracy for random train size 700: 0.9807692307692307


Wnioski do powyższej części:
* Pomimo losowego usuwania próbek z części uczącej, dokładność klasyfikatora wciąż jest wysoka.
* Pomimo nierównego rozkładu dokumentów sklasyfikowanych jako spam oraz nie-spam w części uczącej, klasyfikator wciąż radzi sobie bardzo dobrze.
* Jednak dokładności te są dokładnie taką samą liczbą i nie różnią się chociaż na dalekim miejscu po przecinku, co jest martwiące.
* Problem ten może wynikać z błędów implementacji (prawdopodobnie przy wyliczaniu rozkładu warunkowego wystąpień słów w dokumentach pod warunkiem klasy).